In [3]:
import csv
import json
import pandas as pd

In [6]:
# Define the questions, labels, and response options
data = {
    'Label': [
        'Data Analysis', 'People Interaction', 'Data Entry', 'Problem Solving', 'Innovation', 'Communication', 
        'Teamwork', 'Autonomy', 'Attention to Detail', 'Big Picture Thinking', 'Leadership', 'Technical Skills', 
        'Adaptability', 'Creativity', 'Time Management', 'Process Optimization', 'Mentorship', 'Risk Management', 
        'Project Management', 'Customer Focus', 'Initiative', 'Team Self-Organization', 'Emotional Intelligence', 
        'Conflict Resolution', 'Decision Making', 'Resilience', 'Cultural Awareness', 'Learning Agility'
    ],
    'Question': [
        'How comfortable are you with analyzing datasets to extract insights?',
        'How comfortable are you with working directly with people?',
        'How comfortable are you with completing repetitive tasks like data entry?',
        'How comfortable are you with solving complex problems?',
        'How comfortable are you with coming up with new ideas or solutions?',
        'How comfortable are you with explaining complex ideas to others?',
        'How comfortable are you with collaborating in a team environment?',
        'How comfortable are you with working independently with minimal supervision?',
        'How comfortable are you with focusing on small details to ensure accuracy?',
        'How comfortable are you with considering long-term strategies in your work?',
        'How comfortable are you with leading a team or project?',
        'How comfortable are you with using the technical tools needed for your work?',
        'How comfortable are you with adjusting to new tasks or environments?',
        'How comfortable are you with developing creative solutions to problems?',
        'How comfortable are you with managing your time and meeting deadlines?',
        'How comfortable are you with improving processes or workflows?',
        'How comfortable are you with providing guidance to others?',
        'How comfortable are you with assessing and managing risks?',
        'How comfortable are you with organizing and managing projects?',
        'How comfortable are you with understanding and addressing customer needs?',
        'How comfortable are you with taking the initiative to start tasks without being prompted?',
        'How comfortable are you with organizing a team to meet objectives?',
        'How comfortable are you with recognizing and managing emotions in yourself and others?',
        'How comfortable are you with resolving conflicts within a team?',
        'How comfortable are you with making decisions under pressure?',
        'How comfortable are you with handling setbacks or changes in your work?',
        'How comfortable are you with working with people from diverse backgrounds?',
        'How comfortable are you with quickly learning and applying new skills?'
    ],
    'Response Options': [
        'Not at all comfortable, Slightly comfortable, Moderately comfortable, Very comfortable, Extremely comfortable'
    ] * 28  # Repeat the same response options for all questions
}

# Create the DataFrame
questions_df = pd.DataFrame(data)

# Save to a CSV file
questions_df.to_csv('questions.csv', index=False)

# Provide the path to the CSV file
questions_df.head()

,Label,Question,Response Options
0,Data Analysis,How comfortable are you with analyzing dataset...,"Not at all comfortable, Slightly comfortable, ..."
1,People Interaction,How comfortable are you with working directly ...,"Not at all comfortable, Slightly comfortable, ..."
2,Data Entry,How comfortable are you with completing repeti...,"Not at all comfortable, Slightly comfortable, ..."
3,Problem Solving,How comfortable are you with solving complex p...,"Not at all comfortable, Slightly comfortable, ..."
4,Innovation,How comfortable are you with coming up with ne...,"Not at all comfortable, Slightly comfortable, ..."


In [15]:
import csv
import json

# Function to create a question block in QSF format
def create_question_block(question_id, label, question_text, response_options):
    # Create the response options in the QSF format
    choices = {str(i+1): {"Display": option} for i, option in enumerate(response_options.split(", "))}

    # Create the question block
    question_block = {
        "SurveyID": "SV_xxxx",  # Placeholder for survey ID
        "ElementID": f"QID{question_id}",
        "ElementType": "SQ",  # Single choice question
        "Payload": {
            "QuestionText": question_text,
            "DefaultChoices": False,
            "DataExportTag": label,
            "QuestionType": "MC",  # Multiple Choice
            "Selector": "SAVR",  # Single Answer Vertical
            "SubSelector": "TX",  # Text-based
            "Configuration": {
                "QuestionDescriptionOption": "UseText"
            },
            "QuestionDescription": label,
            "Choices": choices,  # Response options
            "ChoiceOrder": [str(i+1) for i in range(len(choices))],  # Order of choices
            "Validation": {"Type": "None"},
            "GradingData": [],
            "Language": [],
            "NextChoiceId": len(choices) + 1,
            "NextAnswerId": 1
        }
    }
    return question_block

# Load the CSV file with labels, questions, and response options
csv_file = 'questions.csv'  # Replace with your actual CSV file path
questions = []
with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for i, row in enumerate(reader):
        question_id = i + 1  # Increment question ID for each question
        label = row['Label']
        question_text = row['Question']
        response_options = row['Response Options']
        question_block = create_question_block(question_id, label, question_text, response_options)
        questions.append({
            "Type": "BlockElement", 
            "QuestionID": f"QID{question_id}", 
            "ElementID": f"QID{question_id}",
            "ElementType": "SQ",  # Single choice question
            "Payload": question_block['Payload']  # Correctly linking the question payload
        })

# Structure the QSF file format with correct BlockElements linking questions
qsf_structure = {
    "SurveyEntry": {
        "SurveyID": "SV_xxxx",  # Placeholder for survey ID
        "SurveyName": "Team Attribute Evaluation",
        "SurveyOwnerID": "UR_xxxx",
        "SurveyBrandID": "yourBrandID",
        "SurveyStatus": "Active",
        "SurveyStartDate": "2024-01-01T00:00:00Z",
        "SurveyExpirationDate": "2025-01-01T00:00:00Z",
        "SurveyCreationDate": "2024-10-01T00:00:00Z",
        "SurveyLanguage": "EN"
    },
    "SurveyElements": [
        {
            "SurveyID": "SV_xxxx",
            "ElementID": "BL_xxxx",
            "ElementType": "BL",  # Block element
            "Payload": {
                "Type": "Default",
                "Description": "Block 1",
                "BlockElements": [{"Type": "Question", "QuestionID": f"QID{i+1}"} for i in range(len(questions))]
            }
        }
    ] + questions  # Insert all questions as SurveyElements after the block
}

# Save the QSF file as JSON
qsf_file = 'qsf_files/generated_survey.qsf'
with open(qsf_file, 'w') as outfile:
    json.dump(qsf_structure, outfile, indent=2)

print(f"QSF file generated: {qsf_file}")


QSF file generated: qsf_files/generated_survey.qsf


In [19]:
import csv

# Function to create the question block in Advanced Format TXT
def create_question_block(label, question_text, response_options):
    options = response_options.split(", ")
    
    # Start with the question definition and label
    question_block = f"[[Question:MC]]\n[[ID:{label}]]\n{question_text}\n[[Choices]]\n"
    
    # Add the response options as multiple-choice answers
    for option in options:
        question_block += f"{option}\n"
    
    return question_block

# Load the CSV file with labels, questions, and response options
csv_file = 'questions.csv'  # Replace with your actual CSV file path
questions_txt = "[[AdvancedFormat]]\n\n[[Block:Main Block]]\n"

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        label = row['Label']
        question_text = row['Question']
        response_options = row['Response Options']
        
        # Create the question block and append to the overall text
        question_block = create_question_block(label, question_text, response_options)
        questions_txt += question_block + "\n"  # Add newline between questions

questions_txt += "[[Block]]"  # Close the block

# Save the questions to a .txt file in Advanced Format
txt_file = 'generated_survey.txt'
with open(txt_file, 'w') as outfile:
    outfile.write(questions_txt)

print(f"TXT file generated: {txt_file}")


TXT file generated: generated_survey.txt
